In [234]:
from vosk import Model, KaldiRecognizer
import wave
import json

In [235]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from evaluate import load
from scipy.io import wavfile
import noisereduce as nr
from tqdm import tqdm

In [236]:
from sklearn.metrics import f1_score

In [237]:
from vosk import SetLogLevel
SetLogLevel(-2)

In [238]:
np.random.seed(0)

In [239]:
model = Model(lang="ru-RU", model_path="vosk-model-small-ru-0.22")

In [240]:
luga = pd.read_json("rzd/ESC_DATASET_v1.2/annotation/luga.json")

In [241]:
luga

,audio_filepath,id,text,label,attribute
0,21_11_2023/2023_11_21__09_54_58.wav,2023_11_21__09_54_58,назад с башмака,17,-1
1,15_11_2023/2023_11_15__11_38_51.wav,2023_11_15__11_38_51,прекратить зарядку тормозной магистрали,20,-1
2,02_11_2023/2023_11_02__10_41_09.wav,2023_11_02__10_41_09,осадить на двадцать восемь вагонов,4,28
3,15_11_2023/2023_11_15__11_38_41.wav,2023_11_15__11_38_41,зарядка тормозной магистрали,6,-1
4,15_11_2023/2023_11_15__09_41_25.wav,2023_11_15__09_41_25,вышел из межвагонного пространства,7,-1
...,...,...,...,...,...
605,02_11_2023/2023_11_02__10_49_36.wav,2023_11_02__10_49_36,протянуть на восемнадцать вагонов,10,18
606,03_07_2023/3faf9502-846e-11ee-8635-c09bf4619c0...,3faf9502-846e-11ee-8635-c09bf4619c03,осадить на десять вагонов,4,10
607,21_11_2023/2023_11_21__10_38_03.wav,2023_11_21__10_38_03,осадить на восемнадцать вагонов,4,18
608,15_11_2023/2023_11_15__09_45_41.wav,2023_11_15__09_45_41,начать осаживание,3,-1


# Denoising

In [158]:
for row in tqdm(luga.iterrows()):
    path = "rzd/ESC_DATASET_v1.2/luga/" + row[1]['audio_filepath']
    rate, data = wavfile.read(path)
    reduced_noise = nr.reduce_noise(y=data, sr=rate)
    wavfile.write("denoise/luga/" + row[1]['audio_filepath'], rate, reduced_noise)

610it [00:16, 37.94it/s]


In [222]:
import librosa
import soundfile as sf

def get_random_audio_and_remove_silence(file_path, out_path, top_db=12, safe_zone=0.25):
    audio_data, sr = librosa.load(file_path, sr=None)
    non_silent_intervals = librosa.effects.split(audio_data, top_db=top_db)
    
    safe_zone = int(sr*safe_zone)
    unique_indexes = set()
    for start, end in non_silent_intervals:
        unique_indexes.update(range(max(0, start - safe_zone), min(len(audio_data), end + safe_zone)))
    processed_audio = audio_data[list(unique_indexes)]

    sf.write(out_path, processed_audio, sr)

In [123]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import butter, lfilter

def low_pass_filter(data, cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data)
    return y

def reduce_noise(input_wav, output_wav, cutoff=1700.0):
    # Read the input WAV file
    fs, data = wavfile.read(input_wav)
    
    # Apply low-pass filter
    filtered_data = low_pass_filter(data, cutoff, fs)
    
    # Write the filtered data to the output WAV file
    wavfile.write(output_wav, fs, np.int16(filtered_data))

In [96]:
import librosa
from scipy import signal
import soundfile as sf

In [223]:
for row in tqdm(luga.iterrows()):
    in_path = "rzd/ESC_DATASET_v1.2/luga/" + row[1]['audio_filepath']
    out_path = "denoise/luga/" + row[1]['audio_filepath']
    get_random_audio_and_remove_silence(in_path, out_path)

610it [00:04, 126.37it/s]


# Classes

In [20]:
_label2id = {
    "отказ": 0,
    "отмена": 1,
    "подтверждение": 2,
    "начать осаживание": 3,

    "осадить на (количество) вагон": 4,
    "осадить на (количество) вагонов": 4,
    "осадить на (количество) вагона": 4,

    "продолжаем осаживание": 5,
    "зарядка тормозной магистрали": 6,
    "вышел из меж вагонного пространства": 7,
    "продолжаем роспуск": 8,
    "растянуть авто сцепки": 9,
    
    "протянуть на (количество) вагон": 10,
    "протянуть на (количество) вагонов": 10,
    "протянуть на (количество) вагона": 10,

    "от сцепка": 11,
    "назад на башмак": 12,
    "захожу в меж вагонного пространство": 13,
    "остановка": 14,
    "вперёд на башмак": 15,
    "сжать авто сцепки": 16,
    "назад с башмака": 17,
    "тише": 18,
    "вперёд с башмака": 19,
    "прекратить зарядку тормозной магистрали": 20,
    "тормозить": 21,
    "отпустить": 22,
}

In [37]:
labels_dict = {
    "отказ": 0,
    "отмена": 1,
    "подтверждение": 2,
    "начать осаживание": 3,
    "осадить на (количество) вагон": 4,
    "продолжаем осаживание": 5,
    "зарядка тормозной магистрали": 6,
    "вышел из межвагонного пространства": 7,
    "продолжаем роспуск": 8,
    "растянуть автосцепки": 9,
    "протянуть на (количество) вагон": 10,
    "отцепка": 11,
    "назад на башмак": 12,
    "захожу в межвагонное,пространство": 13,
    "остановка": 14,
    "вперед на башмак": 15,
    "сжать автосцепки": 16,
    "назад с башмака": 17,
    "тише": 18,
    "вперед с башмака": 19,
    "прекратить зарядку тормозной магистрали": 20,
    "тормозить": 21,
    "отпустить": 22,
}

_id2label = {
    0: "отказ",
    1: "отмена",
    2: "подтверждение",
    3: "начать осаживание",
    4: "осадить на (количество) вагон",
    5: "продолжаем осаживание",
    6: "зарядка тормозной магистрали",
    7: "вышел из межвагонного пространства",
    8: "продолжаем роспуск",
    9: "растянуть автосцепки",
    10: "протянуть на (количество) вагон",
    11: "отцепка",
    12: "назад на башмак",
    13: "захожу в межвагонное,пространство",
    14: "остановка",
    15: "вперед на башмак",
    16: "сжать автосцепки",
    17: "назад с башмака",
    18: "тише",
    19: "вперед с башмака",
    20: "прекратить зарядку тормозной магистрали",
    21: "тормозить",
    22: "отпустить",
}


In [22]:
numbers_dict = {
    1: "один",
    2: "два",
    3: "три",
    4: "четыре",
    5: "пять",
    6: "шесть",
    7: "семь",
    8: "восемь",
    9: "девять",
    10: "десять",
    11: "одиннадцать",
    12: "двенадцать",
    13: "тринадцать",
    14: "четырнадцать",
    15: "пятнадцать",
    16: "шестнадцать",
    17: "семнадцать",
    18: "восемнадцать",
    19: "девятнадцать",
    20: "двадцать",
    21: "двадцать один",
    22: "двадцать два",
    23: "двадцать три",
    24: "двадцать четыре",
    25: "двадцать пять",
    26: "двадцать шесть",
    27: "двадцать семь",
    28: "двадцать восемь",
    29: "двадцать девять",
    30: "тридцать",
    31: "тридцать один",
    32: "тридцать два",
    33: "тридцать три",
    34: "тридцать четыре",
    35: "тридцать пять",
    36: "тридцать шесть",
    37: "тридцать семь",
    38: "тридцать восемь",
    39: "тридцать девять",
    40: "сорок",
    41: "сорок один",
    42: "сорок два",
    43: "сорок три",
    44: "сорок четыре",
    45: "сорок пять",
    46: "сорок шесть",
    47: "сорок семь",
    48: "сорок восемь",
    49: "сорок девять",
    50: "пятьдесят",
    51: "пятьдесят один",
    52: "пятьдесят два",
    53: "пятьдесят три",
    54: "пятьдесят четыре",
    55: "пятьдесят пять",
    56: "пятьдесят шесть",
    57: "пятьдесят семь",
    58: "пятьдесят восемь",
    59: "пятьдесят девять",
    60: "шестьдесят",
    61: "шестьдесят один",
    62: "шестьдесят два",
    63: "шестьдесят три",
    64: "шестьдесят четыре",
    65: "шестьдесят пять",
    66: "шестьдесят шесть",
    67: "шестьдесят семь",
    68: "шестьдесят восемь",
    69: "шестьдесят девять",
    70: "семьдесят",
    71: "семьдесят один",
    72: "семьдесят два",
    73: "семьдесят три",
    74: "семьдесят четыре",
    75: "семьдесят пять",
    76: "семьдесят шесть",
    77: "семьдесят семь",
    78: "семьдесят восемь",
    79: "семьдесят девять",
    80: "восемьдесят",
    81: "восемьдесят один",
    82: "восемьдесят два",
    83: "восемьдесят три",
    84: "восемьдесят четыре",
    85: "восемьдесят пять",
    86: "восемьдесят шесть",
    87: "восемьдесят семь",
    88: "восемьдесят восемь",
    89: "восемьдесят девять",
    90: "девяносто",
    91: "девяносто один",
    92: "девяносто два",
    93: "девяносто три",
    94: "девяносто четыре",
    95: "девяносто пять",
    96: "девяносто шесть",
    97: "девяносто семь",
    98: "девяносто восемь",
    99: "девяносто девять",
    100: "сто"
}

In [23]:
classes = f'{list(_label2id.keys()) + list(numbers_dict.values()) + ["[unk]"]}'
classes = classes.replace("'", '"')

# Vosk valid on luga

In [252]:
number_dict = {
    "один": 1,
    "два": 2,
    "три": 3,
    "четыре": 4,
    "пять": 5,
    "шесть": 6,
    "семь": 7,
    "восемь": 8,
    "девять": 9,
    "десять": 10,
    "одиннадцать": 11,
    "двенадцать": 12,
    "тринадцать": 13,
    "четырнадцать": 14,
    "пятнадцать": 15,
    "шестнадцать": 16,
    "семнадцать": 17,
    "восемнадцать": 18,
    "девятнадцать": 19,
    "двадцать": 20,
    "тридцать": 30,
    "сорок": 40,
    "пятьдесят": 50,
    "шестьдесят": 60,
    "семьдесят": 70,
    "восемьдесят": 80,
    "девяноста": 90,
    "сто": 100
}

number_list = list(number_dict.keys())

def extract_number(label: list[str]):
    res_list = []
    res_num = 0
    for word in label:
        if word in number_list:
            res_num += number_dict[word]
            res_list.append(word)
    if res_num == 0:
        res_num = -1
    return (res_num, ' '.join(res_list))

def get_vagon_form(n: int) -> str:
    if n % 10 == 1 and n % 100 != 11:
        return "вагон"
    elif 2 <= n % 10 <= 4 and not (12 <= n % 100 <= 14):
        return "вагона"
    else:
        return "вагонов"

vagon_dict = {i: get_vagon_form(i) for i in range(1, 101)}

def postprocess_vagon(str_label: str) -> str:
    label = str_label.split()
    voice_num, voice_str = extract_number(label)
    vagon_form = get_vagon_form(voice_num)

    if label[0] in ['осадить', 'протянуть']:
        label = [label[0]] + ["на"] + [voice_str] + [vagon_form]
    else:
        label = ["на"] + [voice_str] + [vagon_form]

    return ' '.join(label)

['один', 'два', 'три', 'четыре', 'пять', 'шесть', 'семь', 'восемь', 'девять', 'десять', 'одиннадцать', 'двенадцать', 'тринадцать', 'четырнадцать', 'пятнадцать', 'шестнадцать', 'семнадцать', 'восемнадцать', 'девятнадцать', 'двадцать', 'тридцать', 'сорок', 'пятьдесят', 'шестьдесят', 'семьдесят', 'восемьдесят', 'девяноста', 'сто']


In [253]:
extract_number("восемьдесят")

(165, 'восемьдесят семьдесят восемь семь')

In [167]:
from nltk.stem.snowball import SnowballStemmer

correct_id2label = {
    0: "отказ",
    1: "отмена",
    2: "подтверждение",
    3: "начать осаживание",
    4: "осадить на (количество) вагон",
    5: "продолжаем осаживание",
    6: "зарядка тормозной магистрали",
    7: "вышел из межвагонного пространства",
    8: "продолжаем роспуск",
    9: "растянуть автосцепки",
    10: "протянуть на (количество) вагон",
    11: "отцепка",
    12: "назад на башмак",
    13: "захожу в межвагонное пространство",
    14: "остановка",
    15: "вперед на башмак",
    16: "сжать автосцепки",
    17: "назад с башмака",
    18: "тише",
    19: "вперед с башмака",
    20: "прекратить зарядку тормозной магистрали",
    21: "тормозить",
    22: "отпустить",
}

_id2label = {
    "отказ": 0,
    "отмена": 1,
    "подтверждение": 2,
    "начать осаживание": 3,
    "осадить на вагон": 4,
    "продолжаем осаживание": 5,
    "зарядка тормозной магистрали": 6,
    "вышел из меж вагонного пространства": 7,
    "продолжаем роспуск": 8,
    "растянуть авто сцепки": 9,
    "протянуть на вагон": 10,
    "от сцепка": 11,
    "назад на башмак": 12,
    "захожу в меж вагонного пространство": 13,
    "остановка": 14,
    "вперёд на башмак": 15,
    "сжать авто сцепки": 16,
    "назад с башмака": 17,
    "тише": 18,
    "вперёд с башмака": 19,
    "прекратить зарядку тормозной магистрали": 20,
    "тормозить": 21,
    "отпустить": 22,
}
_id2label = dict((v, k) for k, v in _id2label.items())

stemmer = SnowballStemmer("russian") 

filtered_labels = [list(filter(lambda x: x != "(количество)", 
                        _id2label[i].split()))
                    for i in range(23)]
stemmed_labels = [{stemmer.stem(word) for word in label}
                  for label in filtered_labels]

def find_nearest_label(sent: str, stemmed_labels: list[str]) -> tp.Tuple[int, str]:
    processed_sent = {stemmer.stem(word) for word in sent.split()}
    intersects = [len(processed_sent.intersection(label)) for label in stemmed_labels]
    max_common_id = np.argmax(intersects)
    return max_common_id, correct_id2label[max_common_id]

In [231]:
def vosk_asr(path, classes):
    wf = wave.open(path, 'rb')
    rec = KaldiRecognizer(model, wf.getframerate(), classes)
    res = ""
    partial_res = ""
    final_res = ""

    while True:
        data = wf.readframes(10000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res += list(json.loads(rec.Result()).values())[0] + " "
        else:
            partial_res += str(json.loads(rec.PartialResult()))

    final_res = list(json.loads(rec.FinalResult()).values())[0]
    return res, partial_res, final_res

In [169]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# def find_nearest_label(predicted_speech: str, labels: dict) -> str:
#     vectorizer = TfidfVectorizer()
#     tfidf_matrix = vectorizer.fit_transform([predicted_speech] + list(labels.keys()))
#     cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
#     most_similar_index = np.argmax(cosine_similarities)

#     return labels[list(labels.keys())[most_similar_index]]

In [170]:
import resource

peak_ram = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024
print(f'Peak RAM usage: {peak_ram} MB')

Peak RAM usage: 772.71875 MB


In [251]:
preds = []
targets = []
labels = []
pred_targets = []

for row in luga.iterrows():
    path = "rzd/ESC_DATASET_v1.2/luga/" + row[1]['audio_filepath']
    res, partial_res, final_res = vosk_asr(path, classes)
    if res == "":
        res = final_res

    targets += [row[1]['text']]
    labels += [row[1]['label']]
    pred_targets += [find_nearest_label(res, stemmed_labels)[0]]

    if find_nearest_label(res, stemmed_labels)[0] in [4, 10]:
        preds += [postprocess_vagon(res)]
    else:
        preds += [find_nearest_label(res, stemmed_labels)[1]]
    if preds[-1] != targets[-1]:
        print("clear: ", res)
        print("fix:   ", preds[-1], pred_targets[-1], extract_number(preds[-1])[0])
        print("target:", targets[-1])
        print()

clear:  зарядку тормозной магистрали 
fix:    зарядка тормозной магистрали 6 -1
target: прекратить зарядку тормозной магистрали

clear:  осадить на вагонов 
fix:    осадить на  вагонов 4 -1
target: осадить на двадцать шесть вагонов

clear:  двадцать четыре вагона 
fix:    на двадцать четыре вагона 4 26
target: осадить на двадцать четыре вагона

clear:  осадить на вагон 
fix:    осадить на  вагонов 4 -1
target: осадить на один вагон

clear:  
fix:    отказ 0 -1
target: продолжаем роспуск

clear:  
fix:    отказ 0 -1
target: отцепка

clear:  один вагонов 
fix:    на один вагон 4 1
target: осадить на одиннадцать вагонов

clear:  башмака 
fix:    назад на башмак 12 -1
target: вперед с башмака

clear:  отказ 
fix:    отказ 0 -1
target: отпустить

clear:  осадить на вагонов 
fix:    осадить на  вагонов 4 -1
target: осадить на двадцать семь вагонов

clear:  протянуть на вагона
fix:    протянуть на  вагонов 10 -1
target: протянуть на два вагона

clear:  шестнадцать вагонов 
fix:    на шестнадц

In [79]:
import resource

peak_ram = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024
print(f'Peak RAM usage: {peak_ram} MB')

Peak RAM usage: 772.71875 MB


In [30]:
for i in preds:
    print(i)
    print(postprocess_vagon(i))
    break

назад с башмака 
назад  вагонов


In [45]:
post_processing_preds = []
for i in preds:
    post_processing_preds += [find_nearest_label(i, labels_dict)]

In [46]:
f1_score(labels, post_processing_preds, average='weighted')

0.8207299322517616

In [80]:
wer = load("wer")

In [233]:
wer_score = wer.compute(predictions=preds, references=targets)
wer_score

0.08337622233659289

# Whisper valid on luga

In [5]:
import whisper

In [6]:
model_name="base"
lang="ru"

options = whisper.DecodingOptions(language=lang, without_timestamps=True, fp16 = False)
model = whisper.load_model(model_name)
tokenizer = whisper.tokenizer.get_tokenizer(True, language="ru", task=options.task)

In [11]:
import resource

peak_ram = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024
print(f'Peak RAM usage: {peak_ram} MB')

Peak RAM usage: 984.484375 MB


In [16]:
preds = []
targets = []
for row in luga.iterrows():
    path = "rzd/ESC_DATASET_v1.2/luga/" + row[1]['audio_filepath']

    audio = whisper.load_audio(path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio)

    res = model.decode( mel, options).text

    preds += [res]
    targets += [row[1]['text']]
    print(res)
    print(row[1]['text'])
    print()

Назад в ашмаке
назад с башмака

Репротив зарядки, тормозной магистралы.
прекратить зарядку тормозной магистрали

Ассалит на 28 гонах
осадить на двадцать восемь вагонов

Зарядка тормозной магистрали
зарядка тормозной магистрали

Вышел из них в угу на запресс-транса.
вышел из межвагонного пространства

Ротянуть на 16-й вагонах
протянуть на шестнадцать вагонов

Вышел и шмешуагонного пространства.
вышел из межвагонного пространства

Вперед на пасмах!
вперед на башмак

Посадить на 8 вагонах.
осадить на восемь вагонов

осадить на 4 вагона
осадить на четыре вагона

Осадить на 26 гонах.
осадить на двадцать шесть вагонов

Перез башмака
вперед с башмака

Астановка!
остановка

Протянуть на 8 вагонов.
протянуть на восемь вагонов

Протянуть мне 4 гонки.
протянуть на четыре вагона

Ротинут на 17 логинах.
протянуть на семнадцать вагонов

Останавливка!
остановка

Отсюда
отцепка

Посадить на три вагона.
осадить на три вагона

Протенуть на сеймбой грановле.
протянуть на семь вагонов

Протянуть на 16 воб

In [17]:
import resource

peak_ram = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024
print(f'Peak RAM usage: {peak_ram} MB')

Peak RAM usage: 1061.59375 MB


In [20]:
wer_score = wer.compute(predictions=preds, references=targets)
wer_score

0.8008234688625836